In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd drive/MyDrive

/content/drive/MyDrive


In [ ]:
!pip install scikeras

In [ ]:
!pip install openpyxl
import openpyxl

In [ ]:
import os
import shutil
import random
import glob
from PIL import Image
import numpy as np
import pandas as pd
import tensorflow as tf
import numpy as np
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from keras import layers,models
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense ,Dropout
# from keras.wrappers.scikit_learn import KerasClassifier
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from keras.callbacks import EarlyStopping
# from keras.layers.regularization.dropout import Dropout
import gc
import keras.backend as K
import pickle as pkl



In [ ]:
def get_labels(file_name,level):
  count=0
  for i in range(len(file_name)):
    if(count==3):
      break
    if(file_name[i]=='_'):
      count+=1
  i=i-2
  while(file_name[i]!='_'):
    i-=1
  l1=[]
  i=i+1
  while(file_name[i]!='_'):
    l1.append(file_name[i])
    i+=1
  size=len(l1)
  num=0
  for j in range(len(l1)):
    num=num+int(l1[j])*pow(10,size-1)
    size-=1
  #print(num)
  df=pd.read_excel("ftddata3.xlsx")
  #num=num+1
  label= df.at[num,'DX']
  if level==1:
    if label=='CN':
      return label
    elif label=='BV' or label=='SV' or label=='PNFA':
      label='FTD'
      return label
  if level==2:
    if label=='BV':
      return label
    elif label=='SV'or label=='PNFA':
      label='PPA'
      return label
    else:
      return False
  if level==3:
    if label=='SV' or label=='PNFA':
      return label
    else:
      return False




def get_images(path1,level):
  images=[]
  labels=[]
  for files in os.listdir(path1):
    img_path=os.path.join(path1,files)
    if os.path.isfile(img_path):
      img=np.loadtxt(img_path)
      #print
      img=np.array(img)

      label=get_labels(files,level)
      if label:
        labels.append(label)
        images.append(img)
  return np.array(images),np.array(labels)



LEVEL-**1**

In [ ]:
level=2
random_state=1524
path1="our_grid_36_txt"
# random_state=1524
workbook = openpyxl.load_workbook('ftddata3.xlsx')
sheet = workbook['Sheet1']
pkl_done=False

In [ ]:
def get_tt_split(X,y,random_state,test_size):
  X_t, X_test, y_t, y_test = train_test_split( X, y, test_size=test_size, random_state=random_state, stratify=y)
  return X_t,X_test,y_t,y_test

In [ ]:

if(pkl_done):
  X=np.load("jan_Rep1_hier_1_X_pkl", allow_pickle=True)
  y=np.load('jan_Rep1_hier_1_y_pkl', allow_pickle=True)
else:
  X,y=get_images(path1,level)
  fileObject = open("jan_Rep1_hier_3_X_pkl", 'wb')
  fileObject1 = open("jan_Rep1_hier_3_y_pkl", 'wb')
  pkl.dump(X, fileObject)
  pkl.dump(y,fileObject1)
  fileObject.close()
  fileObject1.close()



In [ ]:
X_train, X_test, y_train, y_test = get_tt_split(X,y,random_state,0.15)
X_train,X_val,y_train,y_val=get_tt_split(X_train,y_train,random_state,0.15)

In [ ]:
X_train[0].shape

(71, 142)

In [ ]:
# for i in range(104):
#   if(X[i].shape != (640,640)):
#     print("HI");

In [ ]:
X_val.shape

(20, 71, 142)

In [ ]:
X_test

array([[[  8.5     ,   8.5     ,  42.5     , ..., 255.      ,
         255.      , 255.      ],
        [  0.      ,   8.5     ,   8.5     , ..., 255.      ,
         255.      ,   0.      ],
        [  0.      ,   0.      ,   8.5     , ..., 255.      ,
           0.      ,   0.      ],
        ...,
        [  0.      ,   0.      ,   8.5     , ..., 255.      ,
           0.      ,   0.      ],
        [  0.      ,   8.5     ,   8.5     , ..., 255.      ,
         255.      ,   0.      ],
        [  8.5     ,   8.5     ,  42.5     , ..., 255.      ,
         255.      , 255.      ]],

       [[ 28.333333,  28.333333,   0.      , ..., 170.      ,
         170.      , 170.      ],
        [  0.      ,  28.333333,  28.333333, ..., 170.      ,
         170.      ,   0.      ],
        [  0.      ,   0.      ,  28.333333, ..., 170.      ,
           0.      ,   0.      ],
        ...,
        [  0.      ,   0.      ,  28.333333, ..., 170.      ,
           0.      ,   0.      ],
        [  0

In [ ]:
y_train.shape

(110,)

In [ ]:
y_val

array(['BV', 'PPA', 'PPA', 'BV', 'PPA', 'PPA', 'PPA', 'BV', 'BV', 'PPA',
       'PPA', 'PPA', 'BV', 'BV', 'PPA', 'BV', 'BV', 'BV', 'BV', 'PPA'],
      dtype='<U3')

In [ ]:
y_test

array(['BV', 'BV', 'BV', 'PPA', 'BV', 'PPA', 'BV', 'BV', 'PPA', 'BV',
       'PPA', 'PPA', 'BV', 'BV', 'PPA', 'PPA', 'BV', 'PPA', 'PPA', 'PPA',
       'PPA', 'PPA', 'BV', 'BV'], dtype='<U3')

In [ ]:
X_train=X_train/255
X_test=X_test/255
X_val=X_val/255
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
y_train=le.fit_transform(y_train)
y_val=le.transform(y_val)

In [ ]:
# X_train = tf.convert_to_tensor(X_train, dtype=tf.float32)
# X_test=tf.convert_to_tensor(X_test, dtype=tf.float32)
# X_val= tf.convert_to_tensor(X_val, dtype=tf.float32)

In [ ]:
X_train

array([[[0.01111111, 0.01111111, 0.16666667, ..., 0.33333333,
         0.33333333, 0.33333333],
        [0.        , 0.01111111, 0.01111111, ..., 0.33333333,
         0.33333333, 0.        ],
        [0.        , 0.        , 0.01111111, ..., 0.33333333,
         0.        , 0.        ],
        ...,
        [0.        , 0.        , 0.01111111, ..., 0.33333333,
         0.        , 0.        ],
        [0.        , 0.01111111, 0.01111111, ..., 0.33333333,
         0.33333333, 0.        ],
        [0.01111111, 0.01111111, 0.16666667, ..., 0.33333333,
         0.33333333, 0.33333333]],

       [[0.05555556, 0.05555556, 0.33333333, ..., 0.33333333,
         0.66666667, 0.66666667],
        [0.        , 0.05555556, 0.05555556, ..., 0.66666667,
         0.66666667, 0.        ],
        [0.        , 0.        , 0.05555556, ..., 0.66666667,
         0.        , 0.        ],
        ...,
        [0.        , 0.        , 0.05555556, ..., 0.66666667,
         0.        , 0.        ],
        [0. 

In [ ]:
# y_train = tf.convert_to_tensor(y_train, dtype=tf.int32)

# y_val = tf.convert_to_tensor(y_val, dtype=tf.int32)

In [ ]:
y_train

array([1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0,
       1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0,
       1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1,
       1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0,
       1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1])

In [ ]:
y_val

array([0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1])

In [ ]:
X_train[0].shape[0]

71

In [ ]:
def create_model1(filters,filter_sizes):
  K.clear_session();
  gc.collect();
  model=Sequential()
  for i in range(len(filters)):
        if i == 0:
            model.add(Conv2D(filters[i], filter_sizes[i], activation='relu', input_shape=(X_train[0].shape[0],X_train[0].shape[1],1),padding='SAME'))
        else:
            model.add(Conv2D(filters[i], filter_sizes[i], activation='relu',padding='SAME'))
        model.add(MaxPooling2D(pool_size=(2, 2),padding='SAME'))
  model.add(Flatten())
  model.add(Dense(64,activation='relu'))
  model.add(Dense(1,activation='sigmoid'))
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model
seed_value = 42
np.random.seed(seed_value)
random.seed(seed_value)
tf.random.set_seed(seed_value)
model1=KerasClassifier(model=create_model1,random_state=42,filters=(32,64),filter_sizes=((3,3),(3,3)))
param_grid = {
    'filters': [(32, 64), (64, 128), (128,256),(32,32),(64,64),(128,128)],
    'filter_sizes': [((2,2),(2,2)),((3, 3), (3, 3)), ((5, 5), (5, 5)),((4,4),(4,4))]
}
grid = GridSearchCV(estimator=model1, param_grid=param_grid, cv=3)
grid_result = grid.fit(X_train, y_train)
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

2/2 [==============================] - 0s 112ms/step


3/3 [==============================] - 1s 24ms/step - loss: 0.9334 - accuracy: 0.4110


2/2 [==============================] - 0s 7ms/step


3/3 [==============================] - 1s 33ms/step - loss: 1.4376 - accuracy: 0.5541


4/4 [==============================] - 1s 22ms/step - loss: 0.8435 - accuracy: 0.5182
Best: 0.499750 using {'filter_sizes': ((2, 2), (2, 2)), 'filters': (32, 64)}
0.499750 (0.019465) with: {'filter_sizes': ((2, 2), (2, 2)), 'filters': (32, 64)}
0.499750 (0.019465) with: {'filter_sizes': ((2, 2), (2, 2)), 'filters': (64, 128)}
0.499750 (0.019465) with: {'filter_sizes': ((2, 2), (2, 2)), 'filters': (128, 256)}
0.499750 (0.019465) with: {'filter_sizes': ((2, 2), (2, 2)), 'filters': (32, 32)}
0.499750 (0.019465) with: {'filter_sizes': ((2, 2), (2, 2)), 'filters': (64, 64)}
0.499750 (0.019465) with: {'filter_sizes': ((2, 2), (2, 2)), 'filters': (128, 128)}
0.499750 (0.019465) with: {'filter_sizes': ((3, 3), (3, 3)), 'filters': (32, 64)}
0.499750 (0.019465) with: {'filter_sizes': ((3, 3), (3, 3)), 'filters': (64, 128)}
0.499750 (0.019465) with: {'filter_sizes': ((3, 3), (3, 3)), 'filters': (128, 256)}
0.499750 (0.019465) with: {'filter_sizes': ((3, 3), (3, 3)), 'filters': (32, 32)}
0.499750 

In [ ]:
final_filters_sizes=list(grid_result.best_params_['filter_sizes'])
final_filters=list(grid_result.best_params_['filters'])

In [ ]:
final_filters_sizes=[(2,2),(2,2)]
final_filters=[32,64]

In [ ]:
tf.keras.backend.clear_session()

In [ ]:
def create_model2():
  K.clear_session();
  gc.collect();
  model=Sequential()
  model.add(Conv2D(final_filters[0],final_filters_sizes[0], activation='relu', input_shape=(X_train[0].shape[0],X_train[0].shape[1],1),padding='SAME'))
  model.add(MaxPooling2D(pool_size=(2, 2),padding='SAME'))
  model.add(Conv2D(final_filters[1],final_filters_sizes[1], activation='relu',padding='SAME'))
  model.add(MaxPooling2D(pool_size=(2, 2),padding='SAME'))
  model.add(Flatten())
  model.add(Dense(64,activation='relu'),)
  model.add(Dense(1,activation='sigmoid'))
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model
seed_value = 42
np.random.seed(seed_value)
random.seed(seed_value)
tf.random.set_seed(seed_value)
model2=KerasClassifier(model=create_model2,random_state=42)
param_grid = {
    'batch_size':[8,10,16,20,32],
    'epochs': [5,10,15,20,30,40,50]
}
grid = GridSearchCV(estimator=model2, param_grid=param_grid, cv=3)
grid_result = grid.fit(X_train, y_train)
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))








Streaming output truncated to the last 5000 lines.
Epoch 1/20
10/10 [==============================] - 1s 9ms/step - loss: 0.8733 - accuracy: 0.5676
Epoch 2/20
10/10 [==============================] - 0s 8ms/step - loss: 0.6638 - accuracy: 0.5946
Epoch 3/20
10/10 [==============================] - 0s 8ms/step - loss: 0.6170 - accuracy: 0.7568
Epoch 4/20
10/10 [==============================] - 0s 7ms/step - loss: 0.5089 - accuracy: 0.7973
Epoch 5/20
10/10 [==============================] - 0s 7ms/step - loss: 0.4868 - accuracy: 0.7568
Epoch 6/20
10/10 [==============================] - 0s 8ms/step - loss: 0.3879 - accuracy: 0.8378
Epoch 7/20
10/10 [==============================] - 0s 7ms/step - loss: 0.3155 - accuracy: 0.8784
Epoch 8/20
10/10 [==============================] - 0s 7ms/step - loss: 0.2355 - accuracy: 0.9189
Epoch 9/20
10/10 [==============================] - 0s 7ms/step - loss: 0.2071 - accuracy: 0.9324
Epoch 10/20
10/10 [==============================] - 0s 6ms/step - 

In [ ]:
final_epoch=15
final_batch=20

In [ ]:
final_epoch=(grid_result.best_params_['epochs'])
final_batch=(grid_result.best_params_['batch_size'])

In [ ]:
tf.keras.backend.clear_session()

In [ ]:
def create_model3(optimizer):
  K.clear_session();
  gc.collect();
  model=Sequential()
  model.add(Conv2D(final_filters[0],final_filters_sizes[0], activation='relu', input_shape=(X_train[0].shape[0],X_train[0].shape[1],1),padding='SAME'))
  model.add(MaxPooling2D(pool_size=(2, 2),padding='SAME'))
  model.add(Conv2D(final_filters[1],final_filters_sizes[1], activation='relu',padding='SAME'))
  model.add(MaxPooling2D(pool_size=(2, 2),padding='SAME'))
  model.add(Flatten())
  model.add(Dense(64,activation='relu'),)
  model.add(Dense(1,activation='sigmoid'))
  if optimizer == 'SGD':
        optimizer = 'sgd'
  elif optimizer == 'Adam':
        optimizer = 'adam'
  elif optimizer == 'RMSprop':
        optimizer = 'rmsprop'
  elif optimizer == 'Adagrad':
        optimizer = 'adagrad'
  else:
        optimizer='adadelta'

  model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
  return model
seed_value = 42
np.random.seed(seed_value)
random.seed(seed_value)
tf.random.set_seed(seed_value)
model3=KerasClassifier(model=create_model3,model__optimizer='Adam',epochs=final_epoch,batch_size=final_batch,
                       random_state=42)
param_grid = {
    'optimizer':['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam']
}
grid = GridSearchCV(estimator=model3, param_grid=param_grid, cv=3)
grid_result = grid.fit(X_train, y_train)
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))



Epoch 1/15
4/4 [==============================] - 1s 13ms/step - loss: 0.7319 - accuracy: 0.5890
Epoch 2/15
4/4 [==============================] - 0s 12ms/step - loss: 0.7161 - accuracy: 0.5479
Epoch 3/15
4/4 [==============================] - 0s 12ms/step - loss: 0.6394 - accuracy: 0.5890
Epoch 4/15
4/4 [==============================] - 0s 11ms/step - loss: 0.5862 - accuracy: 0.6575
Epoch 5/15
4/4 [==============================] - 0s 11ms/step - loss: 0.5527 - accuracy: 0.7534
Epoch 6/15
4/4 [==============================] - 0s 11ms/step - loss: 0.5056 - accuracy: 0.7671
Epoch 7/15
4/4 [==============================] - 0s 11ms/step - loss: 0.4534 - accuracy: 0.8493
Epoch 8/15
4/4 [==============================] - 0s 9ms/step - loss: 0.4096 - accuracy: 0.8082
Epoch 9/15
4/4 [==============================] - 0s 10ms/step - loss: 0.4289 - accuracy: 0.7945
Epoch 10/15
4/4 [==============================] - 0s 11ms/step - loss: 0.4071 - accuracy: 0.7945
Epoch 11/15
4/4 [=============

In [ ]:
final_optimizer=grid_result.best_params_['optimizer']

In [ ]:
final_optimizer='Adam'

In [ ]:
tf.keras.backend.clear_session()

In [ ]:
def create_model4(nodes):
  K.clear_session();
  gc.collect();
  model=Sequential()
  model.add(Conv2D(final_filters[0],final_filters_sizes[0], activation='relu', input_shape=(X_train[0].shape[0],X_train[0].shape[1],1),padding='SAME'))
  model.add(MaxPooling2D(pool_size=(2, 2),padding='SAME'))
  model.add(Conv2D(final_filters[1],final_filters_sizes[1], activation='relu',padding='SAME'))
  model.add(MaxPooling2D(pool_size=(2, 2),padding='SAME'))
  model.add(Flatten())
  print(nodes)
  for i in range(len(nodes)):
    model.add(Dense(nodes[i],activation='relu'))


  model.add(Dense(1,activation='sigmoid'))
  model.compile(loss='binary_crossentropy', optimizer=final_optimizer, metrics=['accuracy'])
  return model
seed_value = 42
np.random.seed(seed_value)
random.seed(seed_value)
tf.random.set_seed(seed_value)
model4=KerasClassifier(model=create_model4,nodes=[64], epochs=final_epoch, batch_size=final_batch,random_state=42)
param_grid = {
    'nodes': [[32],[64,32],[128,64,32],[256,128,64,32]]
}
grid = GridSearchCV(estimator=model4, param_grid=param_grid, cv=5)
grid_result = grid.fit(X_train, y_train)
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))







[32]
Epoch 1/15
5/5 [==============================] - 5s 18ms/step - loss: 0.7347 - accuracy: 0.5114
Epoch 2/15
5/5 [==============================] - 0s 11ms/step - loss: 0.6416 - accuracy: 0.6818
Epoch 3/15
5/5 [==============================] - 0s 11ms/step - loss: 0.5327 - accuracy: 0.7386
Epoch 4/15
5/5 [==============================] - 0s 10ms/step - loss: 0.4675 - accuracy: 0.7841
Epoch 5/15
5/5 [==============================] - 0s 11ms/step - loss: 0.4046 - accuracy: 0.8409
Epoch 6/15
5/5 [==============================] - 0s 9ms/step - loss: 0.3901 - accuracy: 0.8523
Epoch 7/15
5/5 [==============================] - 0s 9ms/step - loss: 0.3690 - accuracy: 0.8295
Epoch 8/15
5/5 [==============================] - 0s 9ms/step - loss: 0.3000 - accuracy: 0.9205
Epoch 9/15
5/5 [==============================] - 0s 8ms/step - loss: 0.2651 - accuracy: 0.9318
Epoch 10/15
5/5 [==============================] - 0s 8ms/step - loss: 0.2483 - accuracy: 0.9205
Epoch 11/15
5/5 [============

2/2 [==============================] - 0s 7ms/step
[64, 32]
Epoch 1/15
5/5 [==============================] - 2s 19ms/step - loss: 0.6961 - accuracy: 0.5341
Epoch 2/15
5/5 [==============================] - 0s 11ms/step - loss: 0.6732 - accuracy: 0.6136
Epoch 3/15
5/5 [==============================] - 0s 11ms/step - loss: 0.5589 - accuracy: 0.7045
Epoch 4/15
5/5 [==============================] - 0s 10ms/step - loss: 0.4630 - accuracy: 0.8182
Epoch 5/15
5/5 [==============================] - 0s 10ms/step - loss: 0.3925 - accuracy: 0.8523
Epoch 6/15
5/5 [==============================] - 0s 10ms/step - loss: 0.3392 - accuracy: 0.8977
Epoch 7/15
5/5 [==============================] - 0s 12ms/step - loss: 0.3321 - accuracy: 0.8295
Epoch 8/15
5/5 [==============================] - 0s 9ms/step - loss: 0.2932 - accuracy: 0.8750
Epoch 9/15
5/5 [==============================] - 0s 9ms/step - loss: 0.2334 - accuracy: 0.9432
Epoch 10/15
5/5 [==============================] - 0s 9ms/step - loss

2/2 [==============================] - 0s 7ms/step
[64, 32]
Epoch 1/15
5/5 [==============================] - 1s 13ms/step - loss: 0.7163 - accuracy: 0.5455
Epoch 2/15
5/5 [==============================] - 0s 12ms/step - loss: 0.6222 - accuracy: 0.6591
Epoch 3/15
5/5 [==============================] - 0s 11ms/step - loss: 0.5040 - accuracy: 0.7955
Epoch 4/15
5/5 [==============================] - 0s 11ms/step - loss: 0.4150 - accuracy: 0.8182
Epoch 5/15
5/5 [==============================] - 0s 9ms/step - loss: 0.3605 - accuracy: 0.8523
Epoch 6/15
5/5 [==============================] - 0s 9ms/step - loss: 0.2918 - accuracy: 0.8977
Epoch 7/15
5/5 [==============================] - 0s 9ms/step - loss: 0.3116 - accuracy: 0.8182
Epoch 8/15
5/5 [==============================] - 0s 9ms/step - loss: 0.3405 - accuracy: 0.8636
Epoch 9/15
5/5 [==============================] - 0s 9ms/step - loss: 0.3439 - accuracy: 0.8068
Epoch 10/15
5/5 [==============================] - 0s 9ms/step - loss: 0

In [ ]:
final_nodes=list(grid_result.best_params_['nodes'])

In [ ]:
# final_nodes=[32]

In [ ]:
tf.keras.backend.clear_session()

In [ ]:
# from keras.layers.regularization.dropout import Dropout
def create_model5(dropout):
  K.clear_session();
  gc.collect();
  model=Sequential()
  model.add(Conv2D(final_filters[0],final_filters_sizes[0], activation='relu', input_shape=(X_train[0].shape[0],X_train[0].shape[1],1),padding='SAME'))
  model.add(MaxPooling2D(pool_size=(2, 2),padding='SAME'))
  model.add(Conv2D(final_filters[1],final_filters_sizes[1], activation='relu',padding='SAME'))
  model.add(MaxPooling2D(pool_size=(2, 2),padding='SAME'))
  model.add(Flatten())
  for i in range(len(final_nodes)):
    model.add(Dense(final_nodes[i],activation='relu'))
    model.add(Dropout(rate=dropout))
  model.add(Dense(1,activation='sigmoid'))
  model.compile(loss='binary_crossentropy', optimizer=final_optimizer, metrics=['accuracy'])
  return model
seed_value = 42
np.random.seed(seed_value)
random.seed(seed_value)
tf.random.set_seed(seed_value)
model5=KerasClassifier(model=create_model5,epochs=final_epoch, batch_size=final_batch,random_state=42,dropout=0.1)
param_grid = {
    'dropout': [0.0,0.1,0.2,0.3,0.4,0.5]
}
grid = GridSearchCV(estimator=model5, param_grid=param_grid, cv=3)
grid_result = grid.fit(X_train, y_train)
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))



Epoch 1/15
4/4 [==============================] - 2s 23ms/step - loss: 0.7183 - accuracy: 0.5068
Epoch 2/15
4/4 [==============================] - 0s 14ms/step - loss: 0.7218 - accuracy: 0.5479
Epoch 3/15
4/4 [==============================] - 0s 14ms/step - loss: 0.6476 - accuracy: 0.7260
Epoch 4/15
4/4 [==============================] - 0s 14ms/step - loss: 0.6182 - accuracy: 0.6301
Epoch 5/15
4/4 [==============================] - 0s 13ms/step - loss: 0.5769 - accuracy: 0.7397
Epoch 6/15
4/4 [==============================] - 0s 12ms/step - loss: 0.4798 - accuracy: 0.8356
Epoch 7/15
4/4 [==============================] - 0s 13ms/step - loss: 0.4534 - accuracy: 0.7671
Epoch 8/15
4/4 [==============================] - 0s 12ms/step - loss: 0.3863 - accuracy: 0.8630
Epoch 9/15
4/4 [==============================] - 0s 11ms/step - loss: 0.4265 - accuracy: 0.7808
Epoch 10/15
4/4 [==============================] - 0s 11ms/step - loss: 0.4378 - accuracy: 0.7808
Epoch 11/15
4/4 [============

In [ ]:
final_dropout=(grid_result.best_params_['dropout'])

In [ ]:
tf.keras.backend.clear_session()

In [ ]:
def final_model():
  model=Sequential()
  model.add(Conv2D(final_filters[0],final_filters_sizes[0], activation='relu', input_shape=(X_train[0].shape[0],X_train[0].shape[1],1),padding='SAME'))
  model.add(MaxPooling2D(pool_size=(2, 2),padding='SAME'))
  model.add(Conv2D(final_filters[1],final_filters_sizes[1], activation='relu',padding='SAME'))
  model.add(MaxPooling2D(pool_size=(2, 2),padding='SAME'))
  model.add(Flatten())
  for i in range(len(final_nodes)):
    model.add(Dense(final_nodes[i],activation='relu'),)
    model.add(Dropout(rate=final_dropout))


  model.add(Dense(1,activation='sigmoid'))
  model.compile(loss='binary_crossentropy', optimizer=final_optimizer, metrics=['accuracy'])
  return model
seed_value = 42
np.random.seed(seed_value)
random.seed(seed_value)
tf.random.set_seed(seed_value)
early_stopping = EarlyStopping(monitor='val_loss', patience=3)
cnn=KerasClassifier(build_fn=final_model, epochs=final_epoch, batch_size=final_batch,random_state=42)
cnn.fit(X_train,y_train,validation_data=(X_val,y_val))


/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


Epoch 1/15
6/6 [==============================] - 3s 71ms/step - loss: 0.7048 - accuracy: 0.5818 - val_loss: 0.7831 - val_accuracy: 0.5000
Epoch 2/15
6/6 [==============================] - 0s 21ms/step - loss: 0.7300 - accuracy: 0.5273 - val_loss: 0.6714 - val_accuracy: 0.6000
Epoch 3/15
6/6 [==============================] - 0s 19ms/step - loss: 0.6321 - accuracy: 0.6545 - val_loss: 0.6637 - val_accuracy: 0.6000
Epoch 4/15
6/6 [==============================] - 0s 20ms/step - loss: 0.5187 - accuracy: 0.7636 - val_loss: 0.7496 - val_accuracy: 0.6500
Epoch 5/15
6/6 [==============================] - 0s 20ms/step - loss: 0.4379 - accuracy: 0.8091 - val_loss: 0.8111 - val_accuracy: 0.5000
Epoch 6/15
6/6 [==============================] - 0s 17ms/step - loss: 0.5515 - accuracy: 0.7364 - val_loss: 0.6873 - val_accuracy: 0.6000
Epoch 7/15
6/6 [==============================] - 0s 17ms/step - loss: 0.4194 - accuracy: 0.8364 - val_loss: 0.6428 - val_accuracy: 0.7000
Epoch 8/15
6/6 [===========

KerasClassifier(
	model=None
	build_fn=<function final_model at 0x7f439f6f1f30>
	warm_start=False
	random_state=42
	optimizer=rmsprop
	loss=None
	metrics=None
	batch_size=20
	validation_batch_size=None
	verbose=1
	callbacks=None
	validation_split=0.0
	shuffle=True
	run_eagerly=False
	epochs=15
	class_weight=None
)

In [ ]:
y_test=le.transform(y_test)
# y_test = tf.convert_to_tensor(y_test, dtype=tf.int32)

In [ ]:
y_test

In [ ]:
# cnn.evaluate(X_test,y_test)
y_pred=cnn.predict(X_test)


2/2 [==============================] - 0s 93ms/step


In [ ]:
y_pred

array([0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1,
       0, 0])

In [ ]:
print('Accuracy: {:.6f}'.format(accuracy_score(y_test, y_pred)))

Accuracy: 0.875000


In [ ]:
final_dropout

In [ ]:
import tensorflow as tf

print("TensorFlow version:", tf.__version__)


Integrating MLP with CNN

In [ ]:
model_name="jan_Rep1_final_hier_L2_85_model"+str(random_state)+".h5"
print(model_name)

jan_Rep1_final_hier_L2_85_model1524.h5


In [ ]:
# model_name="jan_Rep1_final_hier_L2_model"+str(random_state)+".h5"
# print(model_name)

In [ ]:
# model_name="jan_Rep1_final_hier_L3_model"+str(random_state)+".h5"
# print(model_name)

In [ ]:
from keras.models import Model
keras_model = cnn.build_fn()
keras_model.save(model_name)
feature_extraction_model = Model(inputs=keras_model.input, outputs=keras_model.layers[4].output)

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
from keras.models import load_model

# Load the Keras model
loaded_model = load_model(model_name)

In [ ]:
from keras.models import Model
feature_extraction_model = Model(inputs=loaded_model.input, outputs=loaded_model.layers[4].output)

In [ ]:
s=feature_extraction_model.predict(X_train)

4/4 [==============================] - 0s 3ms/step


In [ ]:
s.shape

In [ ]:
def get_rowdata(row_num):
  l1=[]
  for row in sheet.iter_rows(min_row=row_num,max_row=row_num,min_col=2,max_col=37, values_only=True):
    for cell_value in row:
      l1.append(cell_value)
  return l1

In [ ]:
def get_rownum(file_name):
  count=0
  for i in range(len(file_name)):
    if(count==3):
      break
    if(file_name[i]=='_'):
      count+=1
  i=i-2
  while(file_name[i]!='_'):
    i-=1
  l1=[]
  i=i+1
  while(file_name[i]!='_'):
    l1.append(file_name[i])
    i+=1
  size=len(l1)
  num=0
  for j in range(len(l1)):
    num=num+int(l1[j])*pow(10,size-1)
    size-=1
  return num


In [ ]:
def concat_cnnfeatures_with_rawdata(path):
  X_list=[]
  y_list=[]
  for files in os.listdir(path):
    # row_data=[]
    row_num=get_rownum(files)
    l2=get_rowdata(row_num+2)
    label=get_labels(files,level)
    img_path=os.path.join(path,files)
    if os.path.isfile(img_path):
      img=np.loadtxt(img_path)
      #print
      img=np.array(img)
      shapes=img.shape
      img=img.reshape(1,shapes[0],shapes[1])
    print(img.shape)
    l3=feature_extraction_model.predict(img)
    l4=l3.tolist()
    for i in range(len(l4[0])):
      l2.append(l4[0][i])
    if label:
      X_list.append(l2)
      y_list.append(label)
  return np.array(X_list),np.array(y_list)

In [ ]:
X,y=concat_cnnfeatures_with_rawdata(path1)

(1, 71, 142)
1/1 [==============================] - 0s 118ms/step
(1, 71, 142)
1/1 [==============================] - 0s 18ms/step
(1, 71, 142)
1/1 [==============================] - 0s 45ms/step
(1, 71, 142)
1/1 [==============================] - 0s 17ms/step
(1, 71, 142)
1/1 [==============================] - 0s 18ms/step
(1, 71, 142)
1/1 [==============================] - 0s 16ms/step
(1, 71, 142)
1/1 [==============================] - 0s 19ms/step
(1, 71, 142)
1/1 [==============================] - 0s 18ms/step
(1, 71, 142)
1/1 [==============================] - 0s 19ms/step
(1, 71, 142)
1/1 [==============================] - 0s 17ms/step
(1, 71, 142)
1/1 [==============================] - 0s 19ms/step
(1, 71, 142)
1/1 [==============================] - 0s 19ms/step
(1, 71, 142)
1/1 [==============================] - 0s 16ms/step
(1, 71, 142)
1/1 [==============================] - 0s 19ms/step
(1, 71, 142)
1/1 [==============================] - 0s 20ms/step
(1, 71, 142)
1/1 [======

In [ ]:
file_x="jan_rep1_MLP_L2_85_X_Pkl"+str(random_state)
file_y="jan_rep1_MLP_L2_85_y_Pkl"+str(random_state)

In [ ]:
fileObject = open(file_x, 'wb')
fileObject1 = open(file_y, 'wb')

In [ ]:
import pickle as pkl

In [ ]:
pkl.dump(X, fileObject)
pkl.dump(y,fileObject1)
fileObject.close()
fileObject1.close()

In [ ]:
file_1 = open(file_x, 'rb')
file_2=open(file_y,'rb')

In [ ]:
X=pkl.load(file_1)
y=pkl.load(file_2)

In [ ]:
y.shape

In [ ]:
y

In [ ]:
X_train, X_test, y_train, y_test = get_tt_split(X,y,random_state,0.15)
X_train,X_val,y_train,y_val=get_tt_split(X_train,y_train,random_state,0.15)

In [ ]:
y_train.shape

In [ ]:
y_test.shape

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val=scaler.transform(X_val)
X_test = scaler.transform(X_test)

In [ ]:
from sklearn.preprocessing import LabelEncoder
le1=LabelEncoder()
y_train=le1.fit_transform(y_train)


In [ ]:
y_val=le1.transform(y_val)
y_test=le1.transform(y_test)

In [ ]:
X_train.shape

In [ ]:
y_train

In [ ]:
y_val

In [ ]:
y_test

In [ ]:
def create_model21():
  K.clear_session();
  gc.collect();
  model=Sequential()

  model.add(Dense(128,activation='relu',input_shape=(X_train.shape[1],)))
  model.add(Dense(1,activation='sigmoid'))
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model
seed_value = 42
np.random.seed(seed_value)
random.seed(seed_value)
tf.random.set_seed(seed_value)
model2=KerasClassifier(build_fn=create_model21,random_state=42)
param_grid = {
    'batch_size':[8,10,16,20,32],
    'epochs': [5,10,15,20,30,40,50]
}
grid2 = GridSearchCV(estimator=model2, param_grid=param_grid, cv=3)
grid_result = grid2.fit(X_train, y_train)
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


Epoch 1/5
10/10 [==============================] - 1s 5ms/step - loss: 8.7904 - accuracy: 0.6712
Epoch 2/5
10/10 [==============================] - 0s 5ms/step - loss: 6.2504 - accuracy: 0.7671
Epoch 3/5
10/10 [==============================] - 0s 5ms/step - loss: 0.8155 - accuracy: 0.9589
Epoch 4/5
10/10 [==============================] - 0s 4ms/step - loss: 0.3069 - accuracy: 0.9589
Epoch 5/5
5/5 [==============================] - 0s 3ms/step


/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


Epoch 1/5
10/10 [==============================] - 1s 7ms/step - loss: 4.5280 - accuracy: 0.7123
Epoch 2/5
10/10 [==============================] - 0s 5ms/step - loss: 2.2637 - accuracy: 0.8767
Epoch 3/5
10/10 [==============================] - 0s 4ms/step - loss: 1.9333 - accuracy: 0.9041
Epoch 4/5
10/10 [==============================] - 0s 4ms/step - loss: 0.1903 - accuracy: 0.9726
Epoch 5/5
5/5 [==============================] - 0s 3ms/step


/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


...

Epoch 1/50
3/3 [==============================] - 1s 14ms/step - loss: 2.5163 - accuracy: 0.6164
Epoch 2/50
3/3 [==============================] - 0s 11ms/step - loss: 0.8276 - accuracy: 0.9315
Epoch 3/50
3/3 [==============================] - 0s 9ms/step - loss: 0.2513 - accuracy: 0.9589
Epoch 4/50
3/3 [==============================] - 0s 9ms/step - loss: 0.0198 - accuracy: 0.9863
Epoch 5/50
3/3 [==============================] - 0s 12ms/step - loss: 0.0824 - accuracy: 0.9863
Epoch 6/50
3/3 [==============================] - 0s 10ms/step - loss: 1.0761e-04 - accuracy: 1.0000
Epoch 7/50
3/3 [==============================] - 0s 12ms/step - loss: 1.1979e-05 - accuracy: 1.0000
Epoch 8/50
3/3 [==============================] - 0s 10ms/step - loss: 1.8309e-05 - accuracy: 1.0000
Epoch 9/50
3/3 [==============================] - 0s 12ms/step - loss: 8.6088e-05 - accuracy: 1.0000
Epoch 10/50
3/3 [==============================] - 0s 10ms/step - loss: 3.8515e-04 - accuracy: 1.0000
Epoch 11/50

/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


Epoch 1/50
3/3 [==============================] - 1s 10ms/step - loss: 4.8402 - accuracy: 0.5946
Epoch 2/50
3/3 [==============================] - 0s 9ms/step - loss: 2.0648 - accuracy: 0.8919
Epoch 3/50
3/3 [==============================] - 0s 7ms/step - loss: 0.8075 - accuracy: 0.9324
Epoch 4/50
3/3 [==============================] - 0s 8ms/step - loss: 0.3542 - accuracy: 0.9595
Epoch 5/50
3/3 [==============================] - 0s 8ms/step - loss: 0.0868 - accuracy: 0.9730
Epoch 6/50
3/3 [==============================] - 0s 9ms/step - loss: 0.0097 - accuracy: 0.9865
Epoch 7/50
3/3 [==============================] - 0s 8ms/step - loss: 0.2339 - accuracy: 0.9595
Epoch 8/50
3/3 [==============================] - 0s 7ms/step - loss: 0.0095 - accuracy: 0.9865
Epoch 9/50
3/3 [==============================] - 0s 8ms/step - loss: 0.0096 - accuracy: 0.9865
Epoch 10/50
3/3 [==============================] - 0s 9ms/step - loss: 0.0104 - accuracy: 0.9865
Epoch 11/50
3/3 [=====================

/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


Epoch 1/10
11/11 [==============================] - 1s 5ms/step - loss: 12.9629 - accuracy: 0.7273
Epoch 2/10
11/11 [==============================] - 0s 4ms/step - loss: 4.5248 - accuracy: 0.8636
Epoch 3/10
11/11 [==============================] - 0s 4ms/step - loss: 2.3016 - accuracy: 0.9000
Epoch 4/10
11/11 [==============================] - 0s 4ms/step - loss: 0.9075 - accuracy: 0.9273
Epoch 5/10
11/11 [==============================] - 0s 4ms/step - loss: 0.4201 - accuracy: 0.9818
Epoch 6/10
11/11 [==============================] - 0s 4ms/step - loss: 0.0066 - accuracy: 1.0000
Epoch 7/10
11/11 [==============================] - 0s 5ms/step - loss: 0.0010 - accuracy: 1.0000
Epoch 8/10
11/11 [==============================] - 0s 4ms/step - loss: 7.8137e-04 - accuracy: 1.0000
Epoch 9/10
11/11 [==============================] - 0s 4ms/step - loss: 5.9419e-04 - accuracy: 1.0000
Epoch 10/10
11/11 [==============================] - 0s 4ms/step - loss: 4.6809e-04 - accuracy: 1.0000
Best: 

In [ ]:

final_epoch=10
final_batch=10

In [ ]:
final_epoch=(grid_result.best_params_['epochs'])
final_batch=(grid_result.best_params_['batch_size'])

In [ ]:
def create_model31(optimizer):
  K.clear_session();
  gc.collect();
  model=Sequential()

  model.add(Dense(128,activation='relu',input_shape=(X_train.shape[1],)))
  model.add(Dense(1,activation='sigmoid'))
  if optimizer == 'SGD':
        optimizer = 'sgd'
  elif optimizer == 'Adam':
        optimizer = 'adam'
  elif optimizer == 'RMSprop':
        optimizer = 'rmsprop'
  elif optimizer == 'Adagrad':
        optimizer = 'adagrad'
  else:
        optimizer='adadelta'

  model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
  return model
seed_value = 42
np.random.seed(seed_value)
random.seed(seed_value)
tf.random.set_seed(seed_value)
model3=KerasClassifier(model=create_model31,model__optimizer='Adam',epochs=final_epoch,batch_size=final_batch,random_state=42)
param_grid = {
    'optimizer':['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam']
}
grid3 = GridSearchCV(estimator=model3, param_grid=param_grid, cv=5)
grid_result = grid3.fit(X_train, y_train)
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

In [ ]:
final_optimizer=grid_result.best_params_['optimizer']

In [ ]:

final_optimizer='Adam'

In [ ]:
def create_model41(nodes):
  K.clear_session();
  gc.collect();
  model=Sequential()

  for i in range(len(nodes)):
    if(i==0):
      model.add(Dense(nodes[i],activation='relu',input_shape=(X_train.shape[1],)))
    else:
      model.add(Dense(nodes[i],activation='relu'))


  model.add(Dense(1,activation='sigmoid'))
  model.compile(loss='binary_crossentropy', optimizer=final_optimizer, metrics=['accuracy'])
  return model
seed_value = 42
np.random.seed(seed_value)
random.seed(seed_value)
tf.random.set_seed(seed_value)
model4=KerasClassifier(model=create_model41,nodes=[64], epochs=final_epoch, batch_size=final_batch,random_state=42)
param_grid = {
    'nodes': [[200],[400,200],[500,250,125],[600,300],[800,200],[800,400,200],[600,300,150]]
}
grid4 = GridSearchCV(estimator=model4, param_grid=param_grid, cv=5)
grid_result = grid4.fit(X_train, y_train)
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))


Epoch 1/10
9/9 [==============================] - 2s 13ms/step - loss: 8.8737 - accuracy: 0.5795
Epoch 2/10
9/9 [==============================] - 0s 10ms/step - loss: 4.6393 - accuracy: 0.8182
Epoch 3/10
9/9 [==============================] - 0s 10ms/step - loss: 0.6082 - accuracy: 0.9545
Epoch 4/10
9/9 [==============================] - 0s 7ms/step - loss: 0.1798 - accuracy: 0.9773
Epoch 5/10
9/9 [==============================] - 0s 7ms/step - loss: 1.2376 - accuracy: 0.9659
Epoch 6/10
9/9 [==============================] - 0s 6ms/step - loss: 9.8089e-05 - accuracy: 1.0000
Epoch 7/10
9/9 [==============================] - 0s 6ms/step - loss: 0.8847 - accuracy: 0.9886
Epoch 8/10
9/9 [==============================] - 0s 6ms/step - loss: 5.9538e-08 - accuracy: 1.0000
Epoch 9/10
9/9 [==============================] - 0s 7ms/step - loss: 2.0897e-07 - accuracy: 1.0000
Epoch 10/10
3/3 [==============================] - 0s 5ms/step
Epoch 1/10
9/9 [==============================] - 1s 6ms/s

In [ ]:
final_nodes=list(grid_result.best_params_['nodes'])

In [ ]:
final_nodes=[800,200]

In [ ]:
# from keras.layers.regularization.dropout import Dropout
def create_model51(dropout):
  K.clear_session();
  gc.collect();
  model=Sequential()

  for i in range(len(final_nodes)):
    if(i==0):
        model.add(Dense(final_nodes[i],activation='relu',input_shape=(X_train.shape[1],)))
        model.add(Dropout(rate=dropout))
    else:
        model.add(Dense(final_nodes[i],activation='relu'))
        model.add(Dropout(rate=dropout))


  model.add(Dense(1,activation='sigmoid'))
  model.compile(loss='binary_crossentropy', optimizer=final_optimizer, metrics=['accuracy'])
  return model
seed_value = 42
np.random.seed(seed_value)
random.seed(seed_value)
tf.random.set_seed(seed_value)
model5=KerasClassifier(model=create_model51, epochs=final_epoch, batch_size=final_batch,random_state=42,dropout=0.1)
param_grid = {
    'dropout': [0.1,0.2,0.3,0.4,0.5]
}
grid5 = GridSearchCV(estimator=model5, param_grid=param_grid, cv=5)
grid_result = grid5.fit(X_train, y_train)
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

In [ ]:
final_dropout=(grid_result.best_params_['dropout'])

In [ ]:
final_dropout=0.2

In [ ]:
def final_model_mlp():
  model=Sequential()

  for i in range(len(final_nodes)):
    if(i==0):
      model.add(Dense(final_nodes[i],activation='relu',input_shape=(X_train.shape[1],)))
      model.add(Dropout(rate=final_dropout))
    else:
      model.add(Dense(final_nodes[i],activation='relu'),)
      model.add(Dropout(rate=final_dropout))


  model.add(Dense(1,activation='sigmoid'))
  model.compile(loss='binary_crossentropy', optimizer=final_optimizer, metrics=['accuracy'])
  return model
seed_value = 42
np.random.seed(seed_value)
random.seed(seed_value)
tf.random.set_seed(seed_value)
early_stopping = EarlyStopping(monitor='val_loss', patience=final_epoch/2)
mlp=KerasClassifier(build_fn=final_model_mlp, epochs=final_epoch, batch_size=final_batch,random_state=42)
mlp.fit(X_train,y_train,validation_data=(X_val,y_val))

In [ ]:
model_name="cnn+MLP_Ver2_model+final_hier_L2_85_Rep1"+str(random_state)+".h5"

In [ ]:
import joblib

In [ ]:
joblib.dump(mlp, model_name)

In [ ]:
loaded_model = joblib.load(model_name)

In [ ]:
y_pred2=loaded_model.predict(X_test)

In [ ]:
print('Accuracy: {:.6f}'.format(accuracy_score(y_test, y_pred2)))

In [ ]:
k1=mlp.build_fn()

In [ ]:
k1.summary()

In [ ]:
k2=loaded_model.build_fn()

In [ ]:
k2.summary()

In [ ]:
y_pred1=mlp.predict(X_test)

In [ ]:
y_test.shape

In [ ]:
print('Accuracy: {:.6f}'.format(accuracy_score(y_test, y_pred1)))

In [ ]:
y_test

In [ ]:
y_test

In [ ]:
y_pred1

In [ ]:
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

# y_pred_classes = np.argmax(y_pred,axis=1)
confusion_mtx = confusion_matrix(y_test, y_pred2)
cm_str = '\n'.join(['\t'.join(map(str, row)) for row in confusion_mtx])

In [ ]:
file_name="jan_hier_l2_ver2_85_cnn+MLP_cf_mtx"+str(random_state)+".txt"
with open(file_name, 'w') as file:
    file.write(cm_str)

In [ ]:
confusion_mtx

In [ ]:
matrix = confusion_matrix(y_test, y_pred2)
matrix.diagonal()/matrix.sum(axis=1)

In [ ]:
from sklearn.metrics import classification_report
report=classification_report(y_test, y_pred2)

In [ ]:
report

In [ ]:
file_name="jan_hier_l2_85_ver2_cnn+MLP_cf_rpt"+str(random_state)+".txt"
with open(file_name, 'w') as file:
    file.write(report)
# f=open(file_name,"w")
# f.write(classification_report(y_test, y_pred))

In [ ]:
y_orig=le1.inverse_transform(y_test)

In [ ]:
y_orig

In [ ]:
# cnn=final(X_train,y_train,final_filters,final_filters_sizes,
#                         final_epoch,final_batch,final_optimizer,final_nodes,final_dropout)

In [ ]:
# def get_tt_split(X,y,random_state,test_size):
#   X_t, X_test, y_t, y_test = train_test_split( X, y, test_size=test_size, random_state=random_state, stratify=y)
#   return X_t,X_test,y_t,y_test

# def create_model1(filters=(32,64),filter_sizes=((3,3),(3,3))):
#   model=Sequential()
#   for i in range(len(filters)):
#         if i == 0:
#             model.add(Conv2D(filters[i], filter_sizes[i], activation='relu', input_shape=(6, 6, 1),padding='SAME'))
#         else:
#             model.add(Conv2D(filters[i], filter_sizes[i], activation='relu',padding='SAME'))
#         model.add(MaxPooling2D(pool_size=(2, 2),padding='SAME'))
#   model.add(Flatten())
#   model.add(Dense(64,activation='relu'),)
#   model.add(Dense(4,activation='Softmax'))
#   model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
#   return model

# def create_model2(final_filters,final_filters_sizes):
#   model=Sequential()
#   model.add(Conv2D(final_filters[0],final_filters_sizes[0], activation='relu', input_shape=(6, 6, 1),padding='SAME'))
#   model.add(MaxPooling2D(pool_size=(2, 2),padding='SAME'))
#   model.add(Conv2D(final_filters[1],final_filters_sizes[1], activation='relu',padding='SAME'))
#   model.add(MaxPooling2D(pool_size=(2, 2),padding='SAME'))
#   model.add(Flatten())
#   model.add(Dense(64,activation='relu'),)
#   model.add(Dense(4,activation='Softmax'))
#   model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
#   return model

# def create_model3(optimizer='adam',final_filters=final_filters,final_filters_sizes=final_filters_sizes):
#   model=Sequential()
#   model.add(Conv2D(final_filters[0],final_filters_sizes[0], activation='relu', input_shape=(6, 6, 1),padding='SAME'))
#   model.add(MaxPooling2D(pool_size=(2, 2),padding='SAME'))
#   model.add(Conv2D(final_filters[1],final_filters_sizes[1], activation='relu',padding='SAME'))
#   model.add(MaxPooling2D(pool_size=(2, 2),padding='SAME'))
#   model.add(Flatten())
#   model.add(Dense(64,activation='relu'),)
#   model.add(Dense(4,activation='Softmax'))
#   if optimizer == 'SGD':
#         optimizer = 'sgd'
#   elif optimizer == 'Adam':
#         optimizer = 'adam'
#   elif optimizer == 'RMSprop':
#         optimizer = 'rmsprop'
#   elif optimizer == 'Adagrad':
#         optimizer = 'adagrad'
#   else:
#         optimizer='adadelta'

#   model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
#   return model

# def create_model4(nodes=[64],final_filters=final_filters,final_filters_sizes=final_filters_sizes,
#                   final_optimizer=final_optimizer):
#   model=Sequential()
#   model.add(Conv2D(final_filters[0],final_filters_sizes[0], activation='relu', input_shape=(6, 6, 1),padding='SAME'))
#   model.add(MaxPooling2D(pool_size=(2, 2),padding='SAME'))
#   model.add(Conv2D(final_filters[1],final_filters_sizes[1], activation='relu',padding='SAME'))
#   model.add(MaxPooling2D(pool_size=(2, 2),padding='SAME'))
#   model.add(Flatten())
#   print(nodes)
#   for i in range(len(nodes)):
#     model.add(Dense(nodes[i],activation='relu'))


#   model.add(Dense(4,activation='Softmax'))
#   model.compile(loss='sparse_categorical_crossentropy', optimizer=final_optimizer, metrics=['accuracy'])
#   return model

# def create_model5(dropout=0.1,final_filters=final_filters,final_filters_sizes=final_filters_sizes,
#                   final_optimizer=final_optimizer,final_nodes=final_nodes):
#   model=Sequential()
#   model.add(Conv2D(final_filters[0],final_filters_sizes[0], activation='relu', input_shape=(6, 6, 1),padding='SAME'))
#   model.add(MaxPooling2D(pool_size=(2, 2),padding='SAME'))
#   model.add(Conv2D(final_filters[1],final_filters_sizes[1], activation='relu',padding='SAME'))
#   model.add(MaxPooling2D(pool_size=(2, 2),padding='SAME'))
#   model.add(Flatten())
#   for i in range(len(final_nodes)):
#     model.add(Dense(final_nodes[i],activation='relu'))
#     model.add(Dropout(rate=dropout))
#   model.add(Dense(4,activation='Softmax'))
#   model.compile(loss='sparse_categorical_crossentropy', optimizer=final_optimizer, metrics=['accuracy'])
#   return model


# def final_model(final_filters=final_filters,final_filters_sizes=final_filters_sizes,
#                   final_optimizer=final_optimizer,final_nodes=final_nodes,final_dropout=final_dropout):
#   model=Sequential()
#   model.add(Conv2D(final_filters[0],final_filters_sizes[0], activation='relu', input_shape=(6, 6, 1),padding='SAME'))
#   model.add(MaxPooling2D(pool_size=(2, 2),padding='SAME'))
#   model.add(Conv2D(final_filters[1],final_filters_sizes[1], activation='relu',padding='SAME'))
#   model.add(MaxPooling2D(pool_size=(2, 2),padding='SAME'))
#   model.add(Flatten())
#   for i in range(len(final_nodes)):
#     model.add(Dense(final_nodes[i],activation='relu'),)
#     model.add(Dropout(rate=final_dropout))


#   model.add(Dense(4,activation='Softmax'))
#   model.compile(loss='sparse_categorical_crossentropy', optimizer=final_optimizer, metrics=['accuracy'])
#   return model



In [ ]:
# def model1(X_train,y_train):
#   seed_value = 42
#   np.random.seed(seed_value)
#   random.seed(seed_value)
#   tf.random.set_seed(seed_value)
#   model=KerasClassifier(build_fn=create_model1)
#   param_grid = {
#       'filters': [(32, 64), (64, 128), (128,256),(32,32),(64,64),(128,128)],
#       'filter_sizes': [((2,2),(2,2)),((3, 3), (3, 3)), ((5, 5), (5, 5)),((4,4),(4,4))]
#   }
#   grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=5)
#   grid_result = grid.fit(X_train, y_train)
#   print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
#   means = grid_result.cv_results_['mean_test_score']
#   stds = grid_result.cv_results_['std_test_score']
#   params = grid_result.cv_results_['params']
#   for mean, stdev, param in zip(means, stds, params):
#       print("%f (%f) with: %r" % (mean, stdev, param))
#   final_filters_sizes=list(grid_result.best_params_['filter_sizes'])
#   final_filters=list(grid_result.best_params_['filters'])
#   return final_filters, final_filters_sizes

# def model2(X_train,y_train,final_filters,final_filters_sizes):
#   seed_value = 42
#   np.random.seed(seed_value)
#   random.seed(seed_value)
#   tf.random.set_seed(seed_value)
#   model=KerasClassifier(build_fn=create_model2,final_filters=final_filters,
#                         final_filters_sizes=final_filters_sizes)
#   param_grid = {
#       'batch_size':[8,10,16,20,32],
#       'epochs': [5,10,15,20,30,40,50]
#   }
#   grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=5)
#   grid_result = grid.fit(X_train, y_train)
#   print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
#   means = grid_result.cv_results_['mean_test_score']
#   stds = grid_result.cv_results_['std_test_score']
#   params = grid_result.cv_results_['params']
#   for mean, stdev, param in zip(means, stds, params):
#       print("%f (%f) with: %r" % (mean, stdev, param))
#   final_epoch=(grid_result.best_params_['epochs'])
#   final_batch=(grid_result.best_params_['batch_size'])
#   return final_epoch,final_batch

# def model3(X_train,y_train,final_filters,final_filters_sizes,final_epoch,final_batch):
#   seed_value = 42
#   np.random.seed(seed_value)
#   random.seed(seed_value)
#   tf.random.set_seed(seed_value)
#   model=KerasClassifier(build_fn=create_model3,epochs=final_epoch,batch_size=final_batch,
#                         final_filters=final_filters,final_filters_sizes=final_filters_sizes)
#   param_grid = {
#       'optimizer':['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam']
#   }
#   grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=5)
#   grid_result = grid.fit(X_train, y_train)
#   print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
#   means = grid_result.cv_results_['mean_test_score']
#   stds = grid_result.cv_results_['std_test_score']
#   params = grid_result.cv_results_['params']
#   for mean, stdev, param in zip(means, stds, params):
#       print("%f (%f) with: %r" % (mean, stdev, param))
#   final_optimizer=grid_result.best_params_['optimizer']
#   return final_optimizer

# def model4(X_train,y_train,final_filters,final_filters_sizes,final_epoch,final_batch,final_optimizer):
#   seed_value = 42
#   np.random.seed(seed_value)
#   random.seed(seed_value)
#   tf.random.set_seed(seed_value)
#   model=KerasClassifier(build_fn=create_model4, epochs=final_epoch, batch_size=final_batch,
#                         final_filters=final_filters,final_filters_sizes=final_filters_sizes,
#                         final_optimizer=final_optimizer)
#   param_grid = {
#       'nodes': [[32],[64,32],[128,64,32],[256,128,64,32]]
#   }
#   grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=5)
#   grid_result = grid.fit(X_train, y_train)
#   print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
#   means = grid_result.cv_results_['mean_test_score']
#   stds = grid_result.cv_results_['std_test_score']
#   params = grid_result.cv_results_['params']
#   for mean, stdev, param in zip(means, stds, params):
#       print("%f (%f) with: %r" % (mean, stdev, param))
#   final_nodes=list(grid_result.best_params_['nodes'])
#   return final_nodes

# def model5(X_train,y_train,final_filters,final_filters_sizes,final_epoch,final_batch,final_optimizer,final_nodes):
#   seed_value = 42
#   np.random.seed(seed_value)
#   random.seed(seed_value)
#   tf.random.set_seed(seed_value)
#   model=KerasClassifier(build_fn=create_model5, epochs=final_epoch, batch_size=final_batch,
#                         final_filters=final_filters,final_filters_sizes=final_filters_sizes,
#                         final_optimizer=final_optimizer,final_nodes=final_nodes)
#   param_grid = {
#       'dropout': [0.0,0.1,0.2,0.3,0.4,0.5]
#   }
#   grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=5)
#   grid_result = grid.fit(X_train, y_train)
#   print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
#   means = grid_result.cv_results_['mean_test_score']
#   stds = grid_result.cv_results_['std_test_score']
#   params = grid_result.cv_results_['params']
#   for mean, stdev, param in zip(means, stds, params):
#       print("%f (%f) with: %r" % (mean, stdev, param))
#   final_dropout=(grid_result.best_params_['dropout'])
#   return final_dropout

# def final(X_train,y_train,final_filters,final_filters_sizes,
#                        final_epoch,final_batch,final_optimizer,final_nodes,final_dropout):
#   seed_value = 42
#   np.random.seed(seed_value)
#   random.seed(seed_value)
#   tf.random.set_seed(seed_value)
#   early_stopping = EarlyStopping(monitor='val_loss', patience=3)
#   cnn=KerasClassifier(build_fn=final_model,epochs=final_epoch, batch_size=final_batch,
#                         final_filters=final_filters,final_filters_sizes=final_filters_sizes,
#                         final_optimizer=final_optimizer,final_nodes=final_nodes, final_dropout=final_dropout)
#   cnn.fit(X_train,y_train,validation_split=0.2, callbacks=[early_stopping])
#   return cnn

In [ ]:
# random_state=0
# f=open("res1.txt","w+")
# acc=[]
# for i in range(1,4):
#   X,y=get_images("prep_data",i)
#   X_train, X_test, y_train, y_test = get_tt_split(X,y,random_state,0.2)
#   X_train=X_train/255
#   X_test=X_test/255
#   from sklearn.preprocessing import LabelEncoder
#   le=LabelEncoder()
#   y_train=le.fit_transform(y_train)
#   final_filters, final_filters_sizes= model1(X_train,y_train)
#   f.write("model-1 completed successfully\n")
#   # f.write("the best parameters are :")
#   # f.write(str(final_filters))
#   final_epoch, final_batch=model2(X_train,y_train,final_filters,final_filters_sizes)
#   f.write("model-2 completed successfully\n")
#   final_optimizer=model3(X_train,y_train,final_filters,final_filters_sizes,final_epoch,final_batch)
#   f.write("model-3 completed successfully\n")
#   final_nodes=model4(X_train,y_train,final_filters,final_filters_sizes,final_epoch,final_batch,final_optimizer)
#   f.write("model-4 completed successfully\n")
#   final_dropout=model5(X_train,y_train,final_filters,final_filters_sizes,
#                        final_epoch,final_batch,final_optimizer,final_nodes)
#   f.write("model-5 completed successfully\n")
#   cnn=final(X_train,y_train,final_filters,final_filters_sizes,
#                        final_epoch,final_batch,final_optimizer,final_nodes,final_dropout)
#   y_test=le.transform(y_test)
#   y_pred=cnn.predict(X_test)
#   val=accuracy_score(y_test, y_pred)
#   acc.append(val)










In [ ]:
# f=open("sat.txt","w")
# f.write("HI\n")
# f.write("SATHWIK\n")

In [ ]:
# f.write("Heloo\n")